In [1]:
#importando bibliotecas
import numpy as np
import pandas as pd
from operator import add
import pyspark
import os.path

# carregar base de dados
fileName = os.path.join('data', 'CommentsApril2018.csv')
df = pd.read_csv(fileName)

/home/Drii/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
import re
def removerPontuacao(texto):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        texto (str): A string.

    Returns:
        str: The cleaned up string.
    """
    return re.sub(r'[^A-Za-z0-9 ]', '', texto).strip().lower()

def data_prep(df):
    
    new_df = df[['articleID','commentBody', 'editorsSelection']]
    new_df['by_words'] = new_df['commentBody'].apply(lambda x: (removerPontuacao(x).split()))
    del new_df['commentBody']
    
    return new_df.values

#new_df.to_csv(r'data/trat_CommentsApril2018.txt', header=None, index=None, sep=',')


In [3]:
def split_comments(row):
    return row['commentBody'].apply(lambda x: x.split())

def take_10_words(words):
    words.split(' ')
    return ''

RDD = sc.parallelize(data_prep(df))

/home/Drii/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
def trat_words(words):
    return words \
                 .map(lambda kv: (kv,1)) \
                 .reduceByKey(add) \
                 .takeOrdered(10, key = lambda x: -x[0])
    

In [6]:
RDD_count = RDD \
                .map(lambda kv: (kv[0], kv[1], kv[2].trat_words()))


In [ ]:
print(RDD.take(5))
